## PyMetaSeem Clase
A continuación se quiere agrupar todo dentro de una clase y poder realizar el proceso para varios archivos al mismo tiempo. Empezando con un solo archivo como prueba de la clase.

Esta clase se compone de cuatro funciones, las cuales:
 * Lee uno o varios archivos `.fasta` los cuales contienen genomas con uno o varios contigs.
 * Corta un read (secuencia genomica) de un tamaño dado y desde una posición aleatoria dada.
 * Crea un diccionario de reads metagenomicos de varios metagenomas dados.
 * Crea un archivo `.fastq` en donde guarda los datos metagenomicos añadiendo le la calidad.

In [ ]:
class PyMetaSeem:
    def __init__(self, File, longitud ,num_reads):
        self.File = File
        self.longitud = longitud
        self.dic = {}
        self.dicReads = {}
        
        #longitud   (ejem = 150 por defauld o agusto del usuario)
        #numero de reads
        
    # funcion para lectura de archivos
    # ingresa un archivo multifasta y arroja una lista con cada secuencia (contig)
    def lectura_genoma(self):
        lista = []
        #with open(File,'r') as f: # con esta funcion podemos abrir un archivo en especifico con ruta completa
        with open(ruta + '/' + self.File,'r') as f:  # con esta funcion podemos leer los nombres de los archivos de la lista 'genomes' y le agrega la ruta para la lectura 
            lines=f.read() # lectura decada linea del archivo
            lines=lines.split('>') # identificador de '>'
            lines=['>'+ x for x in lines[1:]] # lista con cada elemento que comienza con '>'
            for x in lines:
                x1 = x.replace(">","@")
                x2 = x1.replace("\n",",",1) # el primer '\n' se reemplazapor una coma
                x3 = x2.replace("\n","") # los siguientes se quitan
                
                #reemplazar '>' con '@'
                
                lista.append(x3) # lista con un contig en cada elemento y su identificador
            # convertir la lista en un diccionario        
            for x in lista:
                x = x.split(',')
                self.dic[x[0]] = x[1] 
        return(self.dic)  
    
    # funcion para cortar reads, dado una pocision de inicio y una longitud de corte
    def cutout(self,read,i): #fordward
        cropped_read = read[i:i+n_length]
        return(cropped_read) 
    
    
    
    # funcion para cortar reads desde el final, dado una pocision de inicio, una longitud de corte y el tamaño del inserto 
    def cutout2(self,read,i,inserto): #backward 
        # inserto = 400 (tamaño promedio del inserto)
        j = i + inserto
        # j = i+inserto (pocision de inicio de reversa)
        # n_length = 150 (tamaño del read "cortado")
        cropped2_read = read[j-n_length:inserto]  #(quiero comenzar en la pocision j e ir hacia atras)
        return cropped2_read
    
    # crea el reverso del read ya cortado
    def reverso(self,read_cortado):
        temp_list = list(read_cortado)
        temp_list.reverse()
        return ''.join(temp_list)
    
    # crea el reverso complementario del read cortado 
    def reverso_complementario(self,read,i,inserto):
        result = complementario(reverso(cutout2(read,i,n_length,inserto)))
        return (result)



    
    # funcion para cortar los reads aleatoriamente
    # Al ingresar un conjunto de genomas, este nos arroja un metagenoma con varios reads tomados de los genomas iniciales
    # para esto se le entrega el numero de reads deseados por cada genoma y la longitud.
    def reads(self,num_reads):  # num_reads
        contig = 0
        k = 0
        while k < num_reads:
            for key in self.dic:#range(len(diccionario.keys())):
                newkey = key + '_' + str(k) 
                contig = random.choice(list(self.dic.values())) # escoje un contig al azar 
                i = randint(1,(len(contig)-self.longitud)) # toma una pocision de inicio al azar
                self.dicReads[newkey] = cutout(contig,i,self.longitud) # para el contig dado anteriormente, se corta desde la pocision i de lonjitud ln
                # se debe calcular cuantas veces va a escoger cada contig,dependiendo del tamaño del contig inicial      
            k += 1
        return(self.dicReads)
    
    # funcion de creacion archivo .fastq, tomandoel diccionario de reads cortados y la calidad de los reads
    def crea_fastq(diccionario,longitud,file_name):
        file = open(file_name,'wt')
        for key in diccionario: 
            file.write(str(key))
            file.write(str('\n'))
            file.write(str(diccionario[key]))
            file.write(str('\n'))
            file.write(str('+'))
            file.write(str('\n'))
            file.write(str('A'*longitud)) #calidad
            file.write(str('\n'))     
            
            
            
            
        file.close()